report要求
1.preprocess data??
2.choose feature: 单词的出现次数排名
3.train:SCV??可以再找一找分数更高的训练方式
4.evaluate and table: 10-fold 十字交叉验证

In [2]:
###现在有两个问题：
###1.不知道用什么SVM的模型更好
###2.Preprocess data只有split代码
###3.可以想办法提高score

import pandas as pd
import nltk
import random
import operator
import sklearn
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

dataset_full=[]
url="datasets_coursework1/Hateval/hateval.tsv"


def split_data(url):
    #File=open(url, "r")
    #strFile=File.read()
    #File.close()
    #dataset_line=open(url).readlines()
    dataset_line = pd.read_csv(url, sep='\t')
    
    #print(dataset_line[:5])
    ### ~~~~~~split data each line~~~~~~~~ ###
    for index in range(len(dataset_line)):
        dataset_full.append((dataset_line['text'][index],dataset_line['label'][index]))
    #data_frame=pd.DataFrame(dataset_full)
    #print(data_frame.head())   
    
split_data(url)
#print(dataset_full)

lemmatizer = nltk.stem.WordNetLemmatizer()
stopwords=set(nltk.corpus.stopwords.words('english'))
stopwords.add(".")
stopwords.add(",")
stopwords.add("!")
stopwords.add(":")
stopwords.add("?")

# Function to get the list of tokens from a string
def get_list_tokens(string): 
    sentence_split=nltk.tokenize.sent_tokenize(string)
    list_tokens=[]
    for sentence in sentence_split:
        list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)
        for token in list_tokens_sentence:
            list_tokens.append(lemmatizer.lemmatize(token).lower())
    return list_tokens

# Function to get feature vectors
def get_vector_text(list_vocab,string):
    vector_text=np.zeros(len(list_vocab))
    list_tokens_string=get_list_tokens(string) ##Use function 
    for i, word in enumerate(list_vocab):
        if word in list_tokens_string:
        #print (list_tokens_string.count(word))
            vector_text[i]=list_tokens_string.count(word) #Count words
    return vector_text


# Function to retrieve features based on bag of words
def get_vocabulary(training_set, num_features): 
    vocabulary_inner=[]
    dict_word_frequency={}
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])##Use function
        for word in sentence_tokens:
            if word in stopwords: continue
            if word not in dict_word_frequency: dict_word_frequency[word]=1
            else: dict_word_frequency[word]+=1
    sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)[:num_features]  
    for word,frequency in sorted_list:
        vocabulary_inner.append(word)
    return vocabulary_inner

# Function for training our svm classifier
def train_svm_classifier(training_set, vocabulary): 
    X_train=[]
    Y_train=[]
    for instance in training_set:
        vector_instance=get_vector_text(vocabulary,instance[0]) 
        X_train.append(vector_instance)
        Y_train.append(instance[1])
    #test_svm_function(X_train,Y_train) # Parameters Adjustment 
    scaler = StandardScaler() 
    #print(scaler)
    X_scaled = scaler.fit_transform(X_train) #StandScaler Training dataset
    #print(X_scaled)
    #test_svm_function(X_train,Y_train)
    #svm_clf=sklearn.svm.SVC(kernel="rbf",gamma=3.59) #train:99.7%, test:55.8%
    svm_clf=sklearn.svm.SVC(kernel="linear",C=1,gamma='auto') #train:85% test:76%
    #svm_clf=sklearn.svm.SVC(kernel="poly",degree=2,gamma='auto')#train:59% test 58%
    svm_clf.fit(X_scaled,Y_train)
    print("Train data score:"+str(svm_clf.score(X_scaled,Y_train)))
    
    return svm_clf

#learning curve
def test_svm_function(X_train,Y_train):
    gamma_range=np.logspace(0,1,10)
    score=[]
    for i in gamma_range:
        svm_clf=SVC(kernel="rbf",gamma=i,cache_size=5000).fit(X_train,Y_train)
        score.append(svm_clf.score(X_train,Y_train))
    print(max(score),gamma_range[score.index(max(score))])
    plt.plot(gamma_range,score) 
    return gamma_range[score.index(max(score))]

#grid search
def test_svm_function2(X_train,Y_train):
    gamma_range=np.logspace(-10,1,20)
    coef0_range=np.linspace(0,5,10)

    param_grid=dict(gamma=gamma_range,coef0=coef0_range)

    cv=StratifiedShuffleSplit(n_splits=10,test_size=0.3,random_state=420)
    grid=GridSearchCV(SVC(kernel="rbf"),param_grid=param_grid,cv=cv) 
    grid.fit(X_train,Y_train)
    print("The best parameters are %s with a score of %0.5f"%(grid.best_params_,grid.best_score_))

def Kfold():
    table=pd.DataFrame(columns=['precision','recall','f1','accuracy'])
    kf = KFold(n_splits=10)
    print (kf)
    random.shuffle(dataset_full) 
    kf.get_n_splits(dataset_full)
    index=0
    for train_index, test_index in kf.split(dataset_full):
        train_set_fold=[]
        test_set_fold=[]
        accuracy_total=0.0
        index+=1
        for i,instance in enumerate(dataset_full):
            if i in train_index:
                train_set_fold.append(instance)
            else:
                test_set_fold.append(instance)
                
        vocabulary_fold=get_vocabulary(train_set_fold, 1000)
        svm_clf_fold=train_svm_classifier(train_set_fold, vocabulary_fold)
        X_test_fold=[]
        Y_test_fold=[]
        for instance in test_set_fold:
            vector_instance=get_vector_text(vocabulary_fold,instance[0])
            X_test_fold.append(vector_instance)
            Y_test_fold.append(instance[1])
        Y_test_fold_gold=np.asarray(Y_test_fold)
        X_test_fold=np.asarray(X_test_fold)
        Y_test_predictions_fold=svm_clf_fold.predict(X_test_fold)
        print ("Fold completed.")
        ### compute different value ###
        print("Test data score:"+str(svm_clf_fold.score(X_test_fold,Y_test_fold_gold)))
        accuracy_fold=accuracy_score(Y_test_fold_gold, Y_test_predictions_fold)
        accuracy_total+=accuracy_fold             
        precision_fold=precision_score(Y_test_fold_gold, Y_test_predictions_fold, average='macro')
        recall_fold=recall_score(Y_test_fold_gold, Y_test_predictions_fold, average='macro')
        f1_fold=f1_score(Y_test_fold_gold, Y_test_predictions_fold, average='macro')
        ### draw the graph ###
        table=table.append(pd.DataFrame({'precision':precision_fold,'recall':recall_fold,'f1':f1_fold,'accuracy':accuracy_fold},index=[index]))
    average_accuracy=accuracy_total/10
    print ("\nAverage Accuracy: "+str(round(accuracy_fold,3)))
    print (table)
#train_svm_classifier(dataset_full,get_vocabulary(dataset_full, 100))
Kfold()

KFold(n_splits=10, random_state=None, shuffle=False)
Train data score:0.844320987654321
Fold completed.
Test data score:0.7688888888888888
Train data score:0.8485185185185186
Fold completed.
Test data score:0.75
Train data score:0.845679012345679
Fold completed.
Test data score:0.7477777777777778
Train data score:0.8451851851851852
Fold completed.
Test data score:0.7566666666666667
Train data score:0.8462962962962963
Fold completed.
Test data score:0.7588888888888888
Train data score:0.845925925925926
Fold completed.
Test data score:0.7733333333333333
Train data score:0.8492592592592593
Fold completed.
Test data score:0.7588888888888888
Train data score:0.847037037037037
Fold completed.
Test data score:0.7366666666666667
Train data score:0.8461728395061728
Fold completed.
Test data score:0.7411111111111112
Train data score:0.8477777777777777
Fold completed.
Test data score:0.7666666666666667

Average Accuracy: 0.767
    precision    recall        f1  accuracy
1    0.759032  0.760441  0